## Diving Deeper into Pandas

First we will use a simple data set on mouse weights taken from males and females of different strains to illustrate how to obtain simple descriptive statistics, group the data, and plot. 

In [ ]:
import pandas as pd
mousedata = pd.read_csv("data/mouseData.csv")
mousedata

In [ ]:
import matplotlib.pyplot as plt 

##we need this line in our notebook to make matplotlib work with Jupyter
%matplotlib inline

import seaborn as sns

In [ ]:
mousedata.describe()

## Grouping

Another great DataFrame function is groupby(). It will group a DataFrame by one or more columns, and let you iterate through each group. 

In [ ]:
mousedata.groupby(['Gender']).get_group('M')

In [ ]:
## Alternatively:

mousedata[mousedata["Gender"]=="M"]

## Plotting


In [ ]:
# Histogram of weights
sns.distplot(mousedata.Weight)

In [ ]:
# Boxplot
sns.boxplot(x = "Gender", y="Weight", data=mousedata)

Let's make a boxplot showing weights for males and females separately. The easiest way to do this is to create a new DataFrame called mouse_gender that splits male and female weights into separate columns:

(You can also map x and y to boxplot for this - Ted)

In [ ]:
# Boxplot separating male and female
mouse_gender=mousedata.pivot(index='MouseID', columns='Gender', values='Weight')
mouse_gender

In [ ]:
sns.boxplot(data=mouse_gender)

In [ ]:
# To add a y-axis label, assign the boxplot to an object first ("fig") and then use the function set()
fig=sns.boxplot(data=mouse_gender)
# This is an alternative way of doing this
sns.boxplot(x = "Gender", y="Weight", data=mousedata)

# Add label to y-axis:
fig.set(xlabel='Gender', ylabel='Weight')

# Set title with matplotlib
plt.title('Mouse weight by gender')
plt.show()

## Exercise: Create boxplots showing the weight data measured from the 2 different strains, B6 and D2. 

In [ ]:
##Space for your answer here


## A more complicated example

Yesterday you learned a little bit about loading data into Pandas. We're going to do much more manipulation and visualization with Pandas using data taken from Imaris. Imaris is image analysis software with many sophisticated functions. Below is a confocal image taken of inner hair cells stained with antibodies against CtBP2 (a pre-synaptic ribbon marker), GluR2 (a post-synaptic receptor) and MyosinVIIA (which labells the entire hair cell). There are three color channels (red, green, and blue) which indicate the intensity of the staining for CtBP2, GluR2 and MyosinVIIa, respectively.

(insert Brad's image here)

We used Imaris to detect all the "spots" in the CtBP2 (red) channel and compute some statistics about these spots. We've extracted the statistics file from the imaris file just to make things easier. Just know that there are routines to extract this information from the file.

In [ ]:
import pandas as pd

point_stats = pd.read_csv("data/points_statistics.csv")

## Exploring the Imaris Statistics

Because this data file was automatically generated by Imaris, we first need to figure out how it is organized.

We can start taking a look at the first few rows of our summary table using `point_stats.head()`. In general, this is a really good practice to get into; sometimes our data may have a header or not, and we may have loaded the data incorrectly.

In [ ]:
##Show first few rows
point_stats.head()

What are some things we notice? Well, there appear to be some data that describe the entire sample (such as "Total Number of Spots") as well as data for localized points identified by Imaris in the red channel (such as "Area").  

In [ ]:
##show last few rows
point_stats.tail()

In [ ]:
##show dimensions of data frame
point_stats.shape

We can also see that attributes for the various traits describing a given spot (such as "Area" and "Volume") are not columns, but rather listed under the categorical column "Name." If we are curious to see this full list of names, use the unique() function: 

In [ ]:
point_stats.Name.unique()

If we want to take the data as is and run some simple descriptive statistics for each column:

In [ ]:
point_stats.describe()

Notice that Python only computes statistics for variables with numerical values.

ID_Object of -1 designates statistics that describe the entire sample. What if we want to look only at data with ID_Object of -1:

In [ ]:
point_stats[point_stats["ID_Object"]==-1]

Let's look at all of the statistics that were collected for a single spot identified by Imaris.

In [ ]:
point_stats[point_stats["ID_Object"]==1]

Let's look at the raw data for Diameter of spots in the X dimension ("Diameter X"):

In [ ]:
point_stats[point_stats["ID_StatisticsType"]==237].head(20)
#OR
point_stats[point_stats["Name"]=="Diameter X"].head(20)

In [ ]:
#What if you only want to view ID_Object, Value, and Name?
point_stats.iloc[:,[1,3,6]].head(20)
##OR
point_stats.loc[:,["ID_Object", "Value", "Name"]]

## Pivoting

Now let's create a DataFrame that is more intuitive in terms of viewing the statistics Imaris has collected for each identified spot in the red channel. We will call this DataFrame `point_stats_matrix`. To do this, use the `pivot()` function, which reshapes data based on column values.

In [ ]:
point_stats_matrix = point_stats.pivot(index='ID_Object', columns='Name', values='Value')
point_stats_matrix.head()

Remember that the statistics for the entire data set (including "Number of spots per time point" and "Total number of spots") have an ID_Object of -1. Let's remove this row:

In [ ]:
point_stats_matrix = point_stats_matrix.drop(-1)
point_stats_matrix.head(20)

In [ ]:
point_stats_matrix.describe()

## Plotting our DataFrame

Next let's try some simple visualization, starting with a histogram of area measurements for the spots: 

In [ ]:
import matplotlib.pyplot as plt 
sns.distplot(point_stats_matrix.Area)

Now how to add labels to axes?

How about a boxplot of Area values?

In [ ]:
point_stats_matrix.boxplot("Area")

In [ ]:
import seaborn as sns
psm = point_stats_matrix

sns.lmplot(x='Intensity Max X', y='Intensity Max Y', fit_reg=False, data=psm)

## Exercise

Take a look at the help for `lmplot` below. 

In [ ]:
help(sns.lmplot)



In [ ]:
cmap = sns.cubehelix_palette(as_cmap=True)

sns.lmplot(x='Intensity Max X', y='Intensity Max Y', fit_reg=False, data=psm)

In [ ]:
## Faceting

Faceting is one of the most powerful ways of exploring data. 

## Filtering

Next we will discuss filtering, sticking with our area data to keep things simple. The boxplot you created for area shows the median area to be about 3.5. What if we want to create a DataFrame named "large_points" that only includes spots with an area greater than 5?

In [ ]:
large_points=point_stats_matrix[point_stats_matrix.Area >= 5]
large_points.describe()

**Question**: what is the output of `point_stats_matrix.Area >= 5`? Try it out by running the below cell. 
    
How does this help us select the rows we want out of `point_stats_matrix`? 

In [ ]:
point_stats_matrix.Area >= 5

## Computing a new column based on other columns

Pandas gets extremely powerful in that you can add new columns based on calculations from other columns.

## Getting data out

What if you want to save the `point_stats_matrix` DataFrame as its own csv file? Try running the code below. Where did it write the dataset?

In [ ]:
point_stats_matrix.to_csv("data/point_stats-mod.csv")

There is also support for reading and writing Excel files if you need it: http://pandas.pydata.org/pandas-docs/stable/io.html#excel-files

## Exercise

Filter the `points_stats_matrix` dataset to have `Intensity Center X` > 10000 and assign it to `psm10000`. (Because of the spaces, you will have to use points_stats_matrix['Intensity Center X'] to access the column).

Plot a scatter plot of psm10000['Position X'] versus psm10000['Position Y'].

## What you learned today

Congrats for getting this far! You have seen lots of features of Pandas and Seaborne that let you manipulate the data and visualize it. 

1. `group_by`
2. Filtering
3. Boxplots and Scatterplots
4. Faceting
5. Pivoting data